In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:


import numpy as np
import pandas as pd
import math
df = pd.read_csv('train_0_truncated_tail.csv')

df.pop('boost0_timer')
df.pop('boost1_timer')
df.pop('boost2_timer')
df.pop('boost3_timer')
df.pop('boost4_timer')
df.pop('boost5_timer')

df.pop('p0_boost')
df.pop('p1_boost')
df.pop('p2_boost')
df.pop('p3_boost')
df.pop('p4_boost')
df.pop('p5_boost')

df.pop('p0_pos_z')
df.pop('p1_pos_z')
df.pop('p2_pos_z')
df.pop('p3_pos_z')
df.pop('p4_pos_z')
df.pop('p5_pos_z')

df.pop('p0_vel_z')
df.pop('p1_vel_z')
df.pop('p2_vel_z')
df.pop('p3_vel_z')
df.pop('p4_vel_z')
df.pop('p5_vel_z')

df.pop('ball_pos_z')
df.pop('ball_vel_z')

df.tail()



,game_num,event_id,event_time,ball_pos_x,ball_pos_y,ball_vel_x,ball_vel_y,p0_pos_x,p0_pos_y,p0_vel_x,...,p4_vel_x,p4_vel_y,p5_pos_x,p5_pos_y,p5_vel_x,p5_vel_y,player_scoring_next,team_scoring_next,team_A_scoring_within_10sec,team_B_scoring_within_10sec
995,737,737005,-0.473386,11.6650,25.500801,-11.0376,48.485400,36.1764,40.2512,5.5962,...,-0.2856,0.8790,-19.553400,102.059600,10.821200,-0.0028,-1,NaN,0,0
996,737,737005,-0.363688,10.3796,31.147000,-10.9984,48.311802,36.6466,38.1190,5.6754,...,-0.3604,1.0516,-17.852600,101.806200,9.924601,-6.2408,-1,NaN,0,0
997,737,737005,-0.254574,9.1902,36.371002,-10.9620,48.150600,37.1824,35.7382,5.9694,...,-4.7684,11.2826,-16.913599,101.149600,10.650000,-7.6132,-1,NaN,0,0
998,737,737005,-0.145479,8.1868,40.778202,-10.9312,48.016200,37.6878,33.5450,6.1292,...,-4.7726,11.2928,-15.999201,100.512800,11.185200,-7.6824,-1,NaN,0,0
999,737,737005,-0.036366,7.0956,45.572200,-10.8976,47.869800,38.4584,29.2822,2.5992,...,-4.7726,11.2928,-15.150000,99.929405,11.398600,-7.8708,-1,NaN,0,0


In [122]:
# event x_a1 - ball closer to team A goal
# event x_a2 - ball closer to team B goal
# event x_b1 - ball colliding with team A player
# event x_b2 - ball colliding with team B player
# event x_c1 - ball rolling toward team A goal
# event x_c2 - ball rolling toward team B goal
import math
import sys

#how many events are in this game?
unique_kickoffs = df['event_id'].unique()
print("unique kickoffs is={}".format(unique_kickoffs))

#how many snapshots are in each event?
event_counts = df.event_id.value_counts()
print("event_counts is=\n{}".format(event_counts))


event_tables = []

#for each kickoff, let's consider our three events in an event table
for i in range(len(unique_kickoffs)):
  event_table=pd.DataFrame(columns=["game_num", "event_id", "ball_loc", "x_a1", "x_a2", "x_b1", "x_b2", "x_c1", "x_c2", "team_scoring"])
  if(i == 0):
    start = 0
  else:
    start += event_counts[unique_kickoffs[i-1]]
  #for all snapshots in the event, we need to determine if any x_a,x_b,or x_c occurring
  for j in range(start, start+event_counts[unique_kickoffs[i]]-1):

    #look through original dataframe
    
    df_row = []
    df_row_check = []
    
    df_row.insert(0,df['game_num'][j])
    df_row_check.insert(0,'game_num')

    df_row.insert(1,df['event_id'][j])
    df_row_check.insert(1,'event_id')

    #determine which side of field the ball is on
    if(df['ball_pos_x'][j] < 0): 
      df_row.insert(2,'A')
      df_row_check.insert(2,'ball_loc')

    if(df['ball_pos_x'][j] > 0): 
      df_row.insert(2,'B')
      df_row_check.insert(2,'ball_loc')

    if(df['ball_pos_x'][j] == 0): 
      df_row.insert(2,math.nan)
      df_row_check.insert(2,'ball_loc')

    min_ball_goal_dist = 5
    #is ball close to team A goal
      ##assume team A goal at x = -80 
    d_ball_goal_a = math.sqrt((df['ball_pos_x'][j]+80)**2 + df['ball_pos_y'][j]**2)

    
    
    if(d_ball_goal_a <= min_ball_goal_dist):
      df_row.insert(3,'yes')
      df_row_check.insert(3,'x_a1')

    if(d_ball_goal_a > min_ball_goal_dist):
      df_row.insert(3,'no')
      df_row_check.insert(3,'x_a1')


    #is ball close to team B goal
      ##assume team B goal at x = 80
    d_ball_goal_b = math.sqrt((df['ball_pos_x'][j]-80)**2 + df['ball_pos_y'][j]**2)
    
    if(d_ball_goal_b <= min_ball_goal_dist):
      df_row.insert(4,'yes')
      df_row_check.insert(4,'x_a2')

    if(d_ball_goal_b > min_ball_goal_dist):
      df_row.insert(4,'no')
      df_row_check.insert(4,'x_a2')




    #is ball colliding with player

    #nearest_player_idx = index of nearest player
    ##need to compute nearest player
    
    min_pbd = sys.maxsize
    min_pbi = -1
    for n in range(0,6):
      player_str = 'p{ind}'.format(ind = n)
      player_str_x = player_str +  '_pos_x'
      player_str_y = player_str +  '_pos_y'
      currdist = math.sqrt((df['ball_pos_x'][j] - df[player_str_x][j])**2 + (df['ball_pos_y'][j]-df[player_str_y][j])**2)
      if(currdist < min_pbd):
        min_pbd = currdist
        min_pbi = n

    nearest_player_idx = min_pbi
    d_player_ball = min_pbd

    min_ball_player_dist = 5

    player_on_team_a = (nearest_player_idx == 0) or (nearest_player_idx == 1) or (nearest_player_idx == 2)

    player_on_team_b = (nearest_player_idx == 3) or (nearest_player_idx == 4) or (nearest_player_idx == 5)


    #is ball colliding with team A player
    if((d_player_ball <= min_ball_player_dist) and player_on_team_a):
      df_row.insert(5,'yes')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'no')
      df_row_check.insert(6,'x_b2')



    #is ball colliding with team B player
    if((d_player_ball <= min_ball_player_dist) and player_on_team_b):
      df_row.insert(5,'no')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'yes')
      df_row_check.insert(6,'x_b2')


    #no player is currently colliding with ball
    if(d_player_ball > min_ball_player_dist):
      df_row.insert(5,'no')
      df_row_check.insert(5,'x_b1')
      df_row.insert(6,'no')
      df_row_check.insert(6,'x_b2')


    


    #is ball rolling toward player A goal
    if(df['ball_vel_x'][j] < 0):
       df_row.insert(7,'yes')
       df_row_check.insert(7,'x_c1')
       df_row.insert(8,'no')
       df_row_check.insert(8,'x_c2')


    #is ball rolling toward player B goal
    if(df['ball_vel_x'][j] > 0):
       df_row.insert(7,'no')
       df_row_check.insert(7,'x_c2')
       df_row.insert(8,'yes')
       df_row_check.insert(8,'x_c2')


    #ball is not moving
    if(df['ball_vel_x'][j] == 0):
       df_row.insert(7,'no')
       df_row_check.insert(7,'x_c1')
       df_row.insert(8,'no')
       df_row_check.insert(8,'x_c2')

    df_row.insert(9,df['team_scoring_next'][j])
    df_row_check.insert(9,'team_scoring') 
  
    event_table = event_table.append({'game_num': df_row[0],
                                        'event_id' : df_row[1],
                                        'ball_loc': df_row[2],
                                        'x_a1' : df_row[3],
                                        'x_a2' : df_row[4],
                                        'x_b1' : df_row[5],
                                        'x_b2' : df_row[6],
                                        'x_c1' : df_row[7],
                                        'x_c2' : df_row[8],
                                        'team_scoring' : df_row[9]}, ignore_index = True)


  event_tables.append(event_table)

for i in range(len(event_tables)):
  print(event_tables[i])


unique kickoffs is=[737004 737005]
event_counts is=
737004    678
737005    322
Name: event_id, dtype: int64
    game_num event_id ball_loc x_a1 x_a2 x_b1 x_b2 x_c1 x_c2 team_scoring
0        737   737004        A   no   no   no   no   no  yes            B
1        737   737004        A   no   no   no   no   no  yes            B
2        737   737004        A   no   no  yes   no   no  yes            B
3        737   737004        A   no   no   no   no   no  yes            B
4        737   737004        A   no   no   no   no   no  yes            B
..       ...      ...      ...  ...  ...  ...  ...  ...  ...          ...
672      737   737004        A   no   no   no  yes   no  yes            B
673      737   737004        A   no   no   no   no   no  yes            B
674      737   737004        A   no   no   no   no   no  yes            B
675      737   737004        A   no   no   no   no   no  yes            B
676      737   737004        A   no   no   no   no   no  yes            B

[6

In [120]:
freq_tables = dict()
for i in range(len(unique_kickoffs)):
  event_table = event_tables[i]
  row_names=["x_a1", "x_a2", "x_b1", "x_b2", "x_c1", "x_c2"]
  col_names=["num_yes","num_no","P(Yes)","P(No)"]
  data = [[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]]
  data[0][0] = (np.array(event_table['x_a1']) == 'yes').sum()
  data[0][1] = (np.array(event_table['x_a1']) == 'no').sum()
  data[0][2] = data[0][0]/len(event_table['x_a1'])
  data[0][3] = data[0][1]/len(event_table['x_a1'])

  data[1][0] = (np.array(event_table['x_a2']) == 'yes').sum()
  data[1][1] = (np.array(event_table['x_a2']) == 'no').sum()
  data[1][2] = data[1][0]/len(event_table['x_a2'])
  data[1][3] = data[1][1]/len(event_table['x_a2'])


  data[2][0] = (np.array(event_table['x_b1']) == 'yes').sum()
  data[2][1] = (np.array(event_table['x_b1']) == 'no').sum()
  data[2][2] = data[2][0]/len(event_table['x_b1'])
  data[2][3] = data[2][1]/len(event_table['x_b1'])


  data[3][0] = (np.array(event_table['x_b2']) == 'yes').sum()
  data[3][1] = (np.array(event_table['x_b2']) == 'no').sum()
  data[3][2] = data[3][0]/len(event_table['x_b2'])
  data[3][3] = data[3][1]/len(event_table['x_b2'])


  data[4][0] = (np.array(event_table['x_c1']) == 'yes').sum()
  data[4][1] = (np.array(event_table['x_c1']) == 'no').sum()
  data[4][2] = data[4][0]/len(event_table['x_c1'])
  data[4][3] = data[4][1]/len(event_table['x_c1'])

  data[5][0] = (np.array(event_table['x_c2']) == 'yes').sum()
  data[5][1] = (np.array(event_table['x_c2']) == 'no').sum()
  data[5][2] = data[5][0]/len(event_table['x_c2'])
  data[5][3] = data[5][1]/len(event_table['x_c2'])

  freq_table=pd.DataFrame(data, row_names, col_names)
  event_id = unique_kickoffs[i]
  freq_tables[event_id] = (freq_table)

for i in unique_kickoffs:
  print("\nprobability table for event {}:\n{}".format(i,freq_tables[i]))


probability table for event 737004:
      num_yes  num_no    P(Yes)     P(No)
x_a1        3     674  0.004431  0.995569
x_a2        0     677  0.000000  1.000000
x_b1       53     624  0.078287  0.921713
x_b2       73     604  0.107829  0.892171
x_c1      317     360  0.468242  0.531758
x_c2      360     317  0.531758  0.468242

probability table for event 737005:
      num_yes  num_no    P(Yes)     P(No)
x_a1        2     319  0.006231  0.993769
x_a2        3     318  0.009346  0.990654
x_b1       26     295  0.080997  0.919003
x_b2       13     308  0.040498  0.959502
x_c1      150     171  0.467290  0.532710
x_c2      155     166  0.482866  0.517134


In [114]:
unique_kickoffs

array([737004, 737005])

In [ ]:
unique_kickoffs = df['event_id'].unique()
unique_kickoffs[0]

event_counts[unique_kickoffs[i]]


df['game_num'][3]



737

In [ ]:
nearest_player_idx = 0
player_str = 'p{i}'.format(i = nearest_player_idx)
player_str +=  '_pos_x'
df[player_str][0]



-56.4992